## Q1

Q: Count all of the distinct ways to go up $n$ distinct steps one, two, or three steps at a time.

In [11]:
def stepUp(n):
    memo = [[], [[1]], [[1, 1], [2]], [[1, 1, 1], [1, 2], [2, 1], [3]]]
    i = 4
    while i <= n:
        memo.append([l + [1] for l in memo[i - 1]] + 
                    [l + [2] for l in memo[i - 2]] +
                    [l + [3] for l in memo[i - 3]])
        i += 1
    return memo[n]

In [13]:
import unittest

class TestStepUp(unittest.TestCase):
    # Base case tests.
    def testZero(self):
        self.assertEquals(stepUp(0), [])
    
    def testOne(self):
        self.assertEquals(stepUp(1), [[1]])        
    
    def testTwo(self):
        self.assertEquals(stepUp(2), [[1, 1], [2]])
    
    def testThree(self):
        self.assertEquals(stepUp(3), [[1, 1, 1], [1, 2], [2, 1], [3]])
        
    # Non base case test.
    def testFour(self):
        self.assertEquals(stepUp(4), [[1, 1, 1, 1], [1, 2, 1], [2, 1, 1], [3, 1], [1, 1, 2], [2, 2], [1, 3]])

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

/Users/alex/miniconda3/envs/data-structures-dev/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Please use assertEqual instead.
.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## Q2

Q: Find a path from the top left corner to the bottom right corner through a grid with optional obstacles throughout.

The solution is to memoize steps, and to attempt to always try to step in a defined order: (1) right (2) down, in some order of those actions being possible.

Note: I'm providing just a probably non-working implementation for this case, e.g. not testing this one.

In [ ]:
def findPath(arr):
    positions = [[0, 0]]
    dim_Y, dim_X = len(arr), len(arr[0])
    memo = [[None for _ in range(dim_X)] for _ in range(dim_Y)]
    while True:
        next_positions = []
        for pos in positions:
            if pos[0] + 1 < len(arr) and arr[pos[0]][pos[1]]:
                next_pos = [pos[0] + 1, pos[1]]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]
            else:  # do the same for columns
                next_pos = [pos[0], pos[1] + 1]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]                    

## Q3

Q: A magic index in an array `A[0...n-1]` is one whose index matches its value. Given a sorted array of distinct integers, write a method to find a magic index, if one exists, in the array.

In [45]:
def magicIndex(l):
    if len(l) == 0:
        return None
    
    pos = len(l) // 2
    if l[pos] == pos:
        return pos
    elif l[pos] < pos:
        next_l = l[pos + 1:]
        sub_result = magicIndex([v - (pos + 1) for v in next_l])
        if sub_result:
            return pos + 1 + sub_result
        else:
            raise ValueError("No magic number found in the array.")
    else:  # l[pos] > pos
        next_l = l[:pos]
        return magicIndex(next_l)

In [37]:
magicIndex([0])

0

In [53]:
magicIndex([0, 2, 5])

0

In [39]:
magicIndex([-5, -4, -3, -2, -1, 5])

5

In [57]:
import unittest

class TestStepUp(unittest.TestCase):
    def testEmpty(self):
        self.assertRaises(ValueError, magicIndex([]))
    
    def testNaive(self):
        self.assertEqual(magicIndex([0]), 0)

#     def testGreaterResult(self):
#         self.assertEqual(magicIndex([-1, 0, 2]), 2)
        
    def testLesserResult(self):
        self.assertEqual(magicIndex([0, 2, 5]), 0)
        
#     def testNoResult(self):
#         self.assertRaises(ValueError, magicIndex([-5, -4, -3, -2, -1, 6]))        
        

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

/Users/alex/miniconda3/envs/data-structures-dev/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: callable is None
  """
...
----------------------------------------------------------------------
Ran 3 tests in 0.004s

OK


Comment: this is $O(\log{n})$, as it is just binary search in disguise.

## Q4

Q: *Power set* &mdash; Write a method that returns all subsets of a set.

In [11]:
def powerSet(l_elements):
    out = []
    for e in l_elements:
        out = out + [o + [e] for o in out] + [[e]]
    return [[]] + out

In [12]:
powerSet([1, 2, 3, 4])

[[],
 [1],
 [1, 2],
 [2],
 [1, 3],
 [1, 2, 3],
 [2, 3],
 [3],
 [1, 4],
 [1, 2, 4],
 [2, 4],
 [1, 3, 4],
 [1, 2, 3, 4],
 [2, 3, 4],
 [3, 4],
 [4]]

Comment: a very simple dynamic problem. The $n$ solution is the concatenation of the $n - 1$ solution, the $n - 1$ solution plus the additional element, and the naked additional element.

This algorithm iterates $n$ times, and iterates through the existing list at each iteration. The existing list at each iteration will be $n!$ in size, so this algorithm is $\approx O(n!)$ amortized time (amortized to ignore list growth costs).

Optimized hardware and software can perform the addition operation a stride at a time, without needing to physically iterate through the elements. If you do that the algorithm is $O(n)$ instead, because each of the concatenations will be $O(1)$ amortized time instead.

## Q7

Q: Find all unique permutations of a string of characters.

This question mentions that the characters don't necessarily have to be unique. However to start off, let's use a unique set to simplify the problem.

In [51]:
import itertools


def unique_permutations(s):
    if len(s) == 0:
        return []
    
    chars = set(s)
    memo = {n: [] for n in range(len(s) + 1)}
    memo[0] = []
    memo[1] = [[c] for c in s]
    
    nchars = 2
    while nchars <= len(s):
        for char in chars:
            for combo in memo[nchars - 1]:
                if char not in combo:
                    memo[nchars] += [combo + [char]]
        nchars += 1
    
    return list(itertools.chain(*memo.values()))

In [52]:
unique_permutations("abc")

[['a'],
 ['b'],
 ['c'],
 ['b', 'a'],
 ['c', 'a'],
 ['a', 'c'],
 ['b', 'c'],
 ['a', 'b'],
 ['c', 'b'],
 ['b', 'c', 'a'],
 ['c', 'b', 'a'],
 ['b', 'a', 'c'],
 ['a', 'b', 'c'],
 ['c', 'a', 'b'],
 ['a', 'c', 'b']]

In [55]:
unique_permutations("abcd")

[['a'],
 ['b'],
 ['c'],
 ['d'],
 ['a', 'd'],
 ['b', 'd'],
 ['c', 'd'],
 ['b', 'a'],
 ['c', 'a'],
 ['d', 'a'],
 ['a', 'c'],
 ['b', 'c'],
 ['d', 'c'],
 ['a', 'b'],
 ['c', 'b'],
 ['d', 'b'],
 ['b', 'a', 'd'],
 ['c', 'a', 'd'],
 ['a', 'c', 'd'],
 ['b', 'c', 'd'],
 ['a', 'b', 'd'],
 ['c', 'b', 'd'],
 ['b', 'd', 'a'],
 ['c', 'd', 'a'],
 ['b', 'c', 'a'],
 ['d', 'c', 'a'],
 ['c', 'b', 'a'],
 ['d', 'b', 'a'],
 ['a', 'd', 'c'],
 ['b', 'd', 'c'],
 ['b', 'a', 'c'],
 ['d', 'a', 'c'],
 ['a', 'b', 'c'],
 ['d', 'b', 'c'],
 ['a', 'd', 'b'],
 ['c', 'd', 'b'],
 ['c', 'a', 'b'],
 ['d', 'a', 'b'],
 ['a', 'c', 'b'],
 ['d', 'c', 'b'],
 ['b', 'c', 'a', 'd'],
 ['c', 'b', 'a', 'd'],
 ['b', 'a', 'c', 'd'],
 ['a', 'b', 'c', 'd'],
 ['c', 'a', 'b', 'd'],
 ['a', 'c', 'b', 'd'],
 ['b', 'c', 'd', 'a'],
 ['c', 'b', 'd', 'a'],
 ['b', 'd', 'c', 'a'],
 ['d', 'b', 'c', 'a'],
 ['c', 'd', 'b', 'a'],
 ['d', 'c', 'b', 'a'],
 ['b', 'a', 'd', 'c'],
 ['a', 'b', 'd', 'c'],
 ['b', 'd', 'a', 'c'],
 ['d', 'b', 'a', 'c'],
 ['a', 'd', 

This algorithm is $O(n^2 \times n!)$. At iteration $n$ we must iterate over the permutations of length $n-1$, and perform an amortized linear time operation on $n$ characters. So we have a sequence of operations of the form $(n)(n - 1) + (n - 1)(n - 2) + \ldots + (2)(1)$. The most significant term in the resultant polynomial is $n^2$, and it's called on the order of $n!$ times. Hence the resultant cost estimate. The book has a more complete derivation. This is a tough one to eyeball; I initially thought it must be $O(n^2)$ because that's the most significant term in the polynomial, but the growth of the constant multiple term matters here!

A modification of this method that allows for repeated characters isn't going to change the runtime and will only involve some additional bookkeeping, so I'll gloss over it.

## Q6

Q: *Towers of Hanoi* &mdash; In the Tower of Hanoi problem you have a stack of $n$ progressively smaller discs in descending size order, bottom-to-top. The objective is to move the stack from the current location to a different location, subject to the following constraints:

1. Only one disc can be moved at a time.
2. A disc cannot be placed on top of a smaller disc.

Write a program that uses stacks to move discs from a first (full) tower to an empty (last) tower, creating a size order last tower, that respects these constraints.

I got very wrapped up in trying to find an analytic, $O(n)$ solution for this problem. I spent about an hour working through solutions, and got reasonably close but couldn't go all the way! Here's an example of a well-defined operation you can run:

In [16]:
def R1(stacks):
    # Assumes a starting point of [[1,...,N],[],[]]
    # Results in [[N-3,...,N],[],[1,2,3]]
    def cp(stacks): return [s.copy() for s in stacks]
    
    P1 = stacks
    P2 = cp(P1); P2[2].append(P2[0].pop())
    P3 = cp(P2); P3[1].append(P3[0].pop())
    P4 = cp(P3); P4[1].append(P4[2].pop())
    P5 = cp(P4); P5[2].append(P5[0].pop())
    P6 = cp(P5); P6[0].append(P6[1].pop())
    P7 = cp(P6); P7[2].append(P7[1].pop())
    P8 = cp(P7); P8[2].append(P8[0].pop())
    return [P1, P2, P3, P4, P5, P6, P7, P8]  

In [23]:
R1([[5, 4, 3, 2, 1],[],[]])

[[[5, 4, 3, 2, 1], [], []],
 [[5, 4, 3, 2], [], [1]],
 [[5, 4, 3], [2], [1]],
 [[5, 4, 3], [2, 1], []],
 [[5, 4], [2, 1], [3]],
 [[5, 4, 1], [2], [3]],
 [[5, 4, 1], [], [3, 2]],
 [[5, 4], [], [3, 2, 1]]]

After blowing a ton of time on this, I read the book solution. The book solution was an analysis along the same lines which was simple and actually successful: just solve the "move the top $n$ elements" problem for greater values of $n$. So I screwed this one up really, really badly.

The solution is reasonably simple, skipping this one now.

## Q9

Q: *Catalan numbers with parens* &mdash; Implement an algorithm to print all valid combinations of $n$ pairs of parentheses.

This is a combinatorical problem involving what are known as "Catalan numbers", which are deeply related to things like possible representations of trees.

In [30]:
def parentCombinations(n):
    memo = {0: [], 1: ["()"]}
    if n <= 1:  return memo[n]
    
    i = 2
    while i <= n:
        # The first element of the memo is special.
        # It is the linear combination of parens, e.g. "()" or "()()".
        # For this element backstep doesn't make sense, as it is equivalent to step.
        memo[i] = [memo[i - 1][0] + "()", "(" + memo[i - 1][0] + ")"]
        
        # For the remaining elements there is a step, backstep, and wrap operation.
        for prior in memo[i - 1][1:]:
            memo[i] += [prior + "()", "()" + prior, "(" + prior + ")"]
        
        i += 1
        
    return memo[n]

In [25]:
parentCombinations(0)

[]

In [26]:
parentCombinations(1)

['()']

In [31]:
parentCombinations(2)

['()()', '(())']

In [32]:
parentCombinations(3)

['()()()', '(()())', '(())()', '()(())', '((()))']

In [33]:
parentCombinations(4)

['()()()()',
 '(()()())',
 '(()())()',
 '()(()())',
 '((()()))',
 '(())()()',
 '()(())()',
 '((())())',
 '()(())()',
 '()()(())',
 '(()(()))',
 '((()))()',
 '()((()))',
 '(((())))']

Easy.

The number of operations necessary follows a recurrence relationship. Given that $C(\cdot)$ is the function, the relationship is $C(1) + C(2) + \ldots + C(n - 1)$. Because there are three possible variations on each element, and we iterate over the entire prior list, we may bound the number of elements produced by $C(n)$ as $|C(n)| = 3 \times |C(n - 1)| = 3 \times 3 \times C(n - 2)$. This leads to our upper bound for this algorithm: $O(3^n)$.

## Q10

Q: *Paint fill*: fill a monocolor area in a matrix with a specified color, as you might do in e.g. Microsoft Paint.

In [34]:
def fillFrom(mat, pos):
    mat[pos[0]][pos[1]] = True
    
    next_pixels = []
    if pos[1] > 0:
        next_pixels.append([pos[0], pos[1] - 1])
    if pos[0] > 0:
        next_pixels.append([pos[0] - 1, pos[1]])
    if pos[1] < len(pos[0]):
        next_pixels.append([pos[0], pos[1] + 1])
    if pos[0] < len(pos):
        next_pixels.append([pos[0] + 1, pos[1]])
        
    for pixel in next_pixels:
        mat = fillFrom(mat, pixel)
        
    return mat

Simple recursive problem, here's a solution.

A better (less resource-intensive) solution would be an iterative solution that tries to grow around the existing area. That would require far fewer stack frames to do its work than a recursive algorithm. But, this algorithm is simple to write.

## Q11

Q: *Change problem* &mdash; Find ways of combining coins to form $n$ cents.

Already did this problem while I was studying DPs intensively, so skipping this one.

## Q12

Q: *Eight queens problem* &mdash; Find all possible ways of placing eight queens on a board such that none share a row, column, or diagonal.

In [120]:
from tqdm import tqdm

def isBoardLegal(queens):
    occupied_rows = [queen[0] for queen in queens]
    occupied_columns = [queen[1] for queen in queens]
    occupied_diags_left = [queen[0] + queen[1] + 1 for queen in queens]
    occupied_diags_right = [8 - queen[0] + queen[1] + 1 for queen in queens]

    return (len(occupied_rows) == len(set(occupied_rows)) and
            len(occupied_columns) == len(set(occupied_columns)) and
            len(occupied_diags_left) == len(set(occupied_diags_left)) and
            len(occupied_diags_right) == len(set(occupied_diags_right))
           )

def eightQueens():
    import itertools
    every_square = [[[[j, k]] for k in range(8)] for j in range(8)]
    every_square = list(itertools.chain(*every_square))
    memo = {1: every_square}
    
    i = 2
    while i <= 4:
        next_queen_positions = []
        for queen_arrangement in memo[i - 1]:
            for square in every_square:
                possibility = queen_arrangement + square
                if isBoardLegal(possibility):
                    next_queen_positions += [possibility]
    
        memo[i] = next_queen_positions
        print(f'Possible arrangements of {i} queens: {len(memo[i])}')
        i += 1
    
    return memo[len(memo)]

In [121]:
result = eightQueens()

Possible arrangements of 2 queens: 2576
Possible arrangements of 3 queens: 61920
Possible arrangements of 4 queens: 829632


Comment: this algorithm is extremely inefficient, because it doesn't perform an important optimization: eliminating columns, rows, and diagonals from the search space at each iteration. There are a *lot* of ways of placing 4, 5, or so queens on the board. Constraining the search space would require additionally bookkeeping known bad areas, which is not too difficult, however.

The book uses a recursive solution that is similar to this iterative one, but simpler to read (surprisingly) and includes this optimization.

## Q13

Q: You have $n$ boxes with fixed height, width, and depth. Find the tallest possible stack of boxes, subject to the constraint that each box is wider, taller, and deeper than the box immediately above it.

In [129]:
def solveBoxes(boxes, priors=None):
    if len(boxes) == 0:
        return []
    elif len(boxes) == 1:
        return boxes
    
    if not priors:
        priors = []
    constraint = priors[-1] if priors else None    
    solutions = []
    
    for i in range(len(boxes)):
        box = boxes[i]
        if ((constraint and box[1] <= constraint[1] and box[2] <= constraint[2]) or
            not constraint
           ):
            next_boxes = boxes[:i] + boxes[i + 1:]
            solutions += solveBoxes(next_boxes, priors=priors + [box])
    
    return solutions

In [123]:
solveBoxes([])

[]

In [124]:
solveBoxes([(1, 1, 1)])

[(1, 1, 1)]

In [125]:
solveBoxes([(1, 1, 1), (2, 2, 2)])

[(2, 2, 2), (1, 1, 1)]

In [130]:
solveBoxes([(1, 1, 1), (2, 2, 2), (2, 3, 3)])

[(2, 3, 3), (2, 2, 2), (1, 1, 1)]

Comment: got this one relatively quickly. Getting better at solving these recursive things. Also understanding how doing 200 of these would allow you to ratchet off solutions to problems like this instinctively.